# Process BCI Comp II Dataset IV
## By Ollie D'Amico
This is not designed to be a clean or all-encompassing notebook. It is just a quick setup for students to utilize. If this alone is submitted/used without modification, a student can expect a failing grade.

In [ ]:
import numpy as np
from itertools import chain
from scipy.io import loadmat
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from scipy.signal import butter, sosfiltfilt, sosfreqz  
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
def listFlatten(l):
    return list(chain.from_iterable(l))

def dc_correct(x):
    # Like baseline correction except it uses entire epoch mean
    nepochs, ntimes, nchans = x.shape
    bl_2D = np.mean(x, axis=1)
    bl_3D = np.transpose(np.repeat(bl_2D, ntimes).reshape(nepochs, nchans, ntimes), (0, 2, 1))

    return np.subtract(x, bl_3D)

def wm(x, start, end, num_points): # Modified from A2
    # Expects x = (num_obs x num_samples x num_channels)
    num_trials = x.shape[0]
    w = np.round((end-start)/num_points).astype(int)
    y = np.zeros((num_points, x.shape[-1], num_trials))
    for i in range(0, num_points):
        s = start + (w * i)
        e = end   + (w * i)
        y[i, :, :] = np.mean(x[:, s:e, :], 1).T
    
    return y

In [ ]:
# Load mat
path = 'C:/Users/Ollie/Downloads/sp1s_aa.mat'
data = loadmat(path)

# Extract relevant pieces of data (y_test is from the website)
clab = data['clab']
x_train = data['x_train']
y_train = data['y_train'].reshape(-1,)
x_test = data['x_test']
y_test = np.array([1,0,0,0,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0,1,1,0,0,1,0,1,1,1,1,0,0,0,1,0,0,1,1,1,1,1,0,1,1,1,1,0,1,1,1,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,0,1,0,1,1,1,0,1,0,1,1,0,1,0,1,1,0,1,1,0])

# Extract channels into a flattened list
chans = np.array(listFlatten(clab[0]))

Generate average ERPs to make sure our data are loaded in and being processed somewhat correctly

In [ ]:
# isolate C3 for plot
ch = np.where(chans == 'C3')[0][0] 

# right hand movements are y_train == 1
x_train_C3_R = x_train[:, ch, np.where(y_train == 1)]
x_train_C3_L = x_train[:, ch, np.where(y_train == 0)]

# Compute averages
avg_C3_R = np.mean(x_train_C3_R, 2)
avg_C3_L = np.mean(x_train_C3_L, 2)

# Plot (right hand plot should look similar to paper's plot)
fs = 100 # Hz
dt = 1000./100 # msec
times = np.arange(-120-500, -120, dt)
plt.plot(times, avg_C3_R);
plt.plot(times, avg_C3_L);
plt.ylim([-25, 20]);
plt.xlabel('Time (ms)');
plt.ylabel('Amplitude at C3 ($\mu$V)');
plt.title('Average ERPs of Right and Left Hand Movement');

In [ ]:
# Extract features
sdt = np.round(dt).astype(int); # rounded dt so that we can index samples
n_points = 3
win_e = -130
win_s = win_e - 210

# Index-space window start and end
w_s = np.where(times == win_s)[0][0]
w_e = np.where(times == win_e)[0][0]

# Transpose data for the wm function defined above
x_train_ = np.transpose(x_train, (2, 0, 1)) # for WM
x_test_ = np.transpose(x_test, (2, 0, 1)) # for WM

# Filter the data
fs = 100.0                                         
lp = 5.                       
order = 2                       

# Create our filter coefficient as as a second-order section
# Note: by defining 'fs' we don't divide our windows by the Nyquist
sos = butter(order, lp, analog = False, btype = 'low', output = 'sos', fs = fs)

# Apply 5 Hz lowpass
x_train_ = sosfiltfilt(sos, x_train_, axis= 1)
x_test_ = sosfiltfilt(sos, x_test_, axis= 1)

# Remove DC offset from each epoch
x_train_ = dc_correct(x_train_)
x_test_ = dc_correct(x_test_)

# Compute windowed means and flatten for sklearn
x_train_wm = wm(x_train_, w_s, w_e, n_points)
x_train_wm_ = x_train_wm.reshape(-1, len(chans)*n_points) 

x_test_wm = wm(x_test_, w_s, w_e, n_points)
x_test_wm_ = x_test_wm.reshape(-1, len(chans)*n_points)

Let's do some simple ML from A2

**Note:** You should plot the ROC/AUC on the training data. If you don't you will not score well on your final project.

In [ ]:
clf_lsqrs = LinearDiscriminantAnalysis(solver = 'lsqr',  shrinkage = 'auto')
score_lsqrs = cross_val_score(clf_lsqrs.fit(x_train_wm_, y_train), x_train_wm_, y_train, cv = 5)
print(f'Cross val performance: {np.mean(score_lsqrs)}')

In [ ]:
# Overfitting just to get a rough idea of our classifier's performance
clf_lsqrs.score(x_train_wm_, y_train)

In [ ]:
clf = clf_lsqrs.fit(x_train_wm_, y_train)
clf.score(x_test_wm_, y_test)